In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from sklearn.metrics import classification_report

In [ ]:
# Load and preprocess images with labels
def load_images_with_labels(folder_real, folder_fake, size=(128, 128)):
    images = []
    labels = []
    filenames = []

In [ ]:
    for file in os.listdir(folder_real):
        path = os.path.join(folder_real, file)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, size)
            img = img.astype('float32') / 255.0
            images.append(img)
            labels.append(0)  # Real
            filenames.append(file)

In [ ]:
    for file in os.listdir(folder_fake):
        path = os.path.join(folder_fake, file)
        img = cv2.imread(path)
        if img is not None:
            img = cv2.resize(img, size)
            img = img.astype('float32') / 255.0
            images.append(img)
            labels.append(1)  # Fake
            filenames.append(file)

In [ ]:
    return np.array(images), np.array(labels), filenames

In [ ]:
# Paths
train_real_path = 'C:/Users/smowe/OneDrive/Desktop/Data Set 1/train/real'
train_fake_path = 'C:/Users/smowe/OneDrive/Desktop/Data Set 1/train/fake'
test_real_path = 'C:/Users/smowe/OneDrive/Desktop/Data Set 1/test/real'
test_fake_path = 'C:/Users/smowe/OneDrive/Desktop/Data Set 1/test/fake'

In [ ]:
# Load training data (only REAL for training autoencoder)
X_train, _, _ = load_images_with_labels(train_real_path, train_real_path)

In [ ]:
# Define autoencoder
input_img = Input(shape=(128, 128, 3))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

In [ ]:
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

In [ ]:
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(X_train, X_train, epochs=50, batch_size=2, shuffle=True)

In [ ]:
# Load test data (real + fake)
X_test, y_test, test_filenames = load_images_with_labels(test_real_path, test_fake_path)

In [ ]:
# Predict and compute reconstruction errors
X_recon = autoencoder.predict(X_test)
errors = np.mean((X_test - X_recon) ** 2, axis=(1, 2, 3))

In [ ]:
# Set threshold (90th percentile)
threshold = np.percentile(errors, 90)

In [ ]:
# Predict labels
y_pred = (errors > threshold).astype(int)

In [ ]:
# Report
print(classification_report(y_test, y_pred, target_names=["REAL", "FAKE"]))

In [ ]:
# Optional visualization
for i in range(len(X_test)):
    label = "FAKE" if y_pred[i] == 1 else "REAL"
    plt.figure(figsize=(6, 3))
    plt.suptitle(f"{test_filenames[i]} - Error: {errors[i]:.6f} - Predicted: {label}", fontsize=10)
    plt.subplot(1, 2, 1)
    plt.imshow(X_test[i])
    plt.title("Original")
    plt.axis('off')
    plt.subplot(1, 2, 2)
    plt.imshow(X_recon[i])
    plt.title("Reconstructed")
    plt.axis('off')
    plt.show()